In [ ]:
# ===============================================================================================================#
# Copyright 2023 Infosys Ltd.                                                                          #
# Use of this source code is governed by Apache License Version 2.0 that can be found in the LICENSE file or at  #
# http://www.apache.org/licenses/                                                                                #
# ===============================================================================================================#

## UC_30 - Indexing documents to vectordb

#### Import libraries

In [ ]:
import json
import os
import shutil
import infy_dpp_sdk
import infy_fs_utils
from _internal_utils.pipeline_helper import PipelineHelper

#### Set environment variables
<div  style="line-height: 1;">
    <span style="color:Red"><b>NOTE:</b> The Pipeline uses environment variables which needs to be set by the developer.<br>
In production developer needs to set them as required.<br>
In this notebook you can provide them using the below code.<br>
To set or change the value please refer <i>installation.ipynb</i></span>
</div>

In [ ]:
%store -r OPENAI_KEY
os.environ['OPENAI_KEY'] = OPENAI_KEY
%store -r OPENAI_SERVER_URL
os.environ['OPENAI_SERVER_URL']=OPENAI_SERVER_URL
%store -r CUSTOM_EMB_MISTRAL_INFERENCE_URL
os.environ['CUSTOM_EMB_MISTRAL_INFERENCE_URL']=CUSTOM_EMB_MISTRAL_INFERENCE_URL

#### Define configuration file path

In [ ]:
STORAGE_ROOT_PATH = 'C:/DPP/infy_libraries_client/STORAGE'
CONTAINER_ROOT_PATH = 'C:/DPP/infy_libraries_client/CONTAINER'
PIPELINE_INPUT_CONFIG_FILE_PATH = '/data/config/dpp_pipeline_index_input_config.json'

#### Copying files
<div style="line-height: 1;">
<span style="color:Red"><b>NOTE: </b>In this notebook below is used to copy sample files to folders in <i>STORAGE_ROOT_PATH</i>.<br>
In production the data and config files should be kept under respective folders in <i>STORAGE_ROOT_PATH </i>.<br>
</span>
</div>

In [ ]:
curr_data = os.path.abspath('./data')
if not os.path.exists(f'{STORAGE_ROOT_PATH}/data'):
    os.makedirs(f'{STORAGE_ROOT_PATH}/data')
if not os.path.exists(f'{STORAGE_ROOT_PATH}/data/input'):
    os.makedirs(f'{STORAGE_ROOT_PATH}/data/input')     
shutil.copy(f'{curr_data}/sample/input/AR_2022-23_page-14-17.pdf',
            f'{STORAGE_ROOT_PATH}/data/input/AR_2022-23_page-14-17.pdf')
shutil.copy(f'{curr_data}/sample/input/AR_2021-22_page-20-21.pdf',
            f'{STORAGE_ROOT_PATH}/data/input/AR_2021-22_page-20-21.pdf')
shutil.copy(f'{curr_data}/sample/input/invoice2.jpg',
            f'{STORAGE_ROOT_PATH}/data/input/invoice2.jpg')
shutil.copy(f'{curr_data}/sample/input/vr.txt',
            f'{STORAGE_ROOT_PATH}/data/input/vr.txt')
shutil.copy(f'{curr_data}/sample/input/sports_statistics.pdf',
            f'{STORAGE_ROOT_PATH}/data/input/sports_statistics.pdf')
shutil.copytree(f'{curr_data}/sample/config',f'{STORAGE_ROOT_PATH}/data/config',
                dirs_exist_ok=True)

In [ ]:
# Show pipeline card
PipelineHelper(PIPELINE_INPUT_CONFIG_FILE_PATH, STORAGE_ROOT_PATH, \
               CONTAINER_ROOT_PATH).show_pipeline_card()

#### Initialize Client Config

In [ ]:
storage_config_data = infy_fs_utils.data.StorageConfigData(
        **{
            "storage_root_uri": f"file://{STORAGE_ROOT_PATH}",
            "storage_server_url": "",
            "storage_access_key": "",
            "storage_secret_key": ""
        })

file_sys_handler = infy_fs_utils.provider.FileSystemHandler(
    storage_config_data)
if not infy_fs_utils.manager.FileSystemManager().has_fs_handler(
    infy_dpp_sdk.common.Constants.FSH_DPP):
    infy_fs_utils.manager.FileSystemManager().add_fs_handler(
        file_sys_handler,
        infy_dpp_sdk.common.Constants.FSH_DPP)

In [ ]:
# Configure client properties
client_config_data = infy_dpp_sdk.ClientConfigData(
    **{
        "container_data": {
            "container_root_path": f"{CONTAINER_ROOT_PATH}",
        }
    })
infy_dpp_sdk.ClientConfigManager().load(client_config_data)

#### Initialize Logging

In [ ]:
import logging
# Modify as required to control the overall logging level
logging.basicConfig(level=logging.ERROR)
logging_config_data = infy_fs_utils.data.LoggingConfigData(
        **{
            # "logger_group_name": "my_group_1",
            "logging_level": 40,
            "logging_format": "",
            "logging_timestamp_format": "",
            "log_file_data": {
                "log_file_dir_path": "/logs",
                "log_file_name_prefix": "index",
                # "log_file_name_suffix": "1",
                "log_file_extension": ".log"

            }})

In [ ]:
if not infy_fs_utils.manager.FileSystemLoggingManager().has_fs_logging_handler(
    infy_dpp_sdk.common.Constants.FSLH_DPP):
    infy_fs_utils.manager.FileSystemLoggingManager().add_fs_logging_handler(
        infy_fs_utils.provider.FileSystemLoggingHandler(
                logging_config_data, file_sys_handler),
            infy_dpp_sdk.common.Constants.FSLH_DPP)

### Run the pipeline

<div  style="line-height: 1;">
    <span style="color:Red"><b>NOTE: </b> Please ignore `No module named 'torch'` or `No module named 'detectron2'` warning.</span>
</div>

In [ ]:
dpp_orchestrator = infy_dpp_sdk.orchestrator.OrchestratorNative(
        input_config_file_path=PIPELINE_INPUT_CONFIG_FILE_PATH)

In [ ]:
processor_response_list = dpp_orchestrator.run_batch()

### Verify results

<div  style="line-height: 1;">
    <span style="color:Green"><b>NOTE: </b> The results of the pipeline will be available in <i>processor_response_data.json</i> file at <i>work_folder_path</i>.</span></div>

In [ ]:
print(json.dumps(processor_response_list[0].
                 context_data.get('request_closer'),indent=4))

<div  style="line-height: 1;">
    <span style="color:Green"><b>NOTE:</b> The vectordb path generated as part of this pipeline is given below.</span></div>

In [ ]:
print(json.dumps(processor_response_list[0].context_data.get(
    "data_encoder"),indent=4))

### Further Processing
<div  style="line-height: 1;">
    <span style="color:Green"><b>NOTE:</b> If you want to run another pipeline as a continuation of this pipeline keep note of <i>group_request_file</i> below, this will be needed in other usecases.</span>
</div>

In [ ]:
print(json.dumps(processor_response_list[0].context_data.get("request_creator"),indent=4))